# Logistic Regression

## Numerical Optimization
The Logistic Regression model is obtained by **minimizing the average cross-entropy between the model predictions and the observed labels**. As we have seen, this corresponds also to a **Maximum Likelihood solution for the observed labels**, or to **minimizing the average Logistic Loss function.** See the next section for all the theory related to Logistic Regression, here we just report the final formulas for the three objective functions:
- log-likelihood: $l(\mathbf{w}, b) = \sum_{i=1}^{n} \left[c_i \log y_i + (1 - c_i) \log(1 - y_i)\right] \rightarrow$ GOAL: maximize $l(\mathbf{w}, b)$ wrt $\mathbf{w}, b$.

- average cross-entroy: $\mathcal{J}(\mathbf{w}, b) = -l(\mathbf{w}, b) = - \sum_{i=1}^{n} \left[c_i \log y_i + (1 - c_i) \log(1 - y_i)\right] \rightarrow$ GOAL: minimize $\mathcal{J}(\mathbf{w}, b)$ wrt $\mathbf{w}, b$.

- average Logistic Loss function: $\mathcal{J}(\mathbf{w}, b) = \sum_{i=1}^{n} \log(1 + e^{-z_i(\mathbf{w}^T \mathbf{x}_i + b)})  \rightarrow$ GOAL: minimize $\mathcal{J}(\mathbf{w}, b)$ wrt $\mathbf{w}, b$.

While for Gaussian models closed form expressions are available for the
ML solutions, this is not the case for Logistic Regression. This means, **we can't just solve system of equations to find the optimal parameters**. This is because the **sigmoid** function involved in binary Logistic regression (and the **softmax** function involved in multiclass Logistic regression) make the loss function nonlinear and non-convex in general<br>
Therefore, we turn to numerical optimization
to find the maximizer of the class likelihoods, or, equivalently, the minimizer of the average cross-entropy or average Logistic Loss function. <br>
Numerical optimization algorithms look for the minimum of a function $f(x)$ with respect to the argument
$x$. Here we briefly explain two methods, the second one will be the one adopted by us:
### 1) Gradient Descent (GD)
with this iterative method, at each iteration $t$ we compute $x_{t+1}$ from $x_{t}$:
- we compute the gradient $\nabla f(x_t)$ of the loss function with respect to the current parameters $x_t$.
- we then update the parameters by moving in the **opposite direction of the gradient** (this is done by multiplying the gradient by $-1$), scaled by a learning rate (also called step) $\alpha_t$:

$$
x_{t+1} = x_t - \alpha_t \nabla f(x_t)
$$

Under the assumptions that the step becomes lower when iterations pass $\left( \alpha_t \rightarrow 0 \right)$ and that the whole sum of all the steps at each iteration is unbounded $\left( \sum_{t=1}^{\infty} \alpha_t \rightarrow \infty \right)$, we are certain that the algorithm converges to a **local minimum** of $f$.
#### Pros of GD
- Easy to implement  
- Low memory usage

#### Cons of GD
- Can be **very slow to converge**  
- Sensitive to choice of learning rate  
- Struggles with ill-conditioned loss surfaces

### 2) L-BFGS (Limited-memory Broyden–Fletcher–Goldfarb–Shanno)
L-BFGS is a more advanced optimization algorithm that uses an **approximate second-order method**. Instead of relying solely on the gradient, it also uses curvature information so second order information, such as the Hessian of the function, from previous iterations to guide the search more efficiently. <br>

#### Pros of L-BFGS
- Much **faster convergence** than gradient descent  
- No need to compute or store the full Hessian (which would be $\mathcal{O}(d^2)$)

#### Cons of L-BFGS
- Slightly more complex and higher per-iteration cost ($\mathcal{O}(md)$, whereas GD has just $\mathcal{O}(d)$)


This second algorithm is the one we''l use and is implemented in `scipy` (requires importing `scipy.optimize`). We will use the `scipy.optimize.fmin_l_bfgs_b` interface to the numerical solver.

`scipy.optimize.fmin_l_bfgs_b` requires at least 2 arguments (check the documentation for more details):

* `func`: the function we want to minimize.
* `x0`: the starting value for the algorithm.

The L-BFGS algorithm requires computing the objective function and its gradient. To pass the gradient we have different options:

* Through `func`: `func` should return a tuple `(f(x), \nabla_x f(x))`.
* Through the optional parameter `fprime`: `fprime` is a function computing the gradient. In this case, `func` should only return the objective value $f(x)$.
* Let the implementation compute an approximated gradient: pass `approx_grad = True`. In this case, `func` should only return the objective value $f(x)$.

The last option does not require writing a function that computes the gradient, as an approximation of the gradient is automatically obtained through finite differences. While this has the advantage that we do not need to derive and implement the gradient, it has two drawbacks:

* The gradient computed through finite differences may not be accurate enough.
* The computations are much more expensive, since we need to evaluate the objective function a number of times at least $D$, where $D$ is the size of $x$, at each iteration, and if we want a more accurate approximation of the gradient we may need to evaluate $f$ many more times.



As an example, we now try to apply the L-BFGS to the function:
$$
f(y, z) = (y + 3)^2 + \sin(y) + (z + 1)^2
$$

In [9]:
import numpy as np
import scipy.optimize as opt

In [10]:
#implementation of function f(y,z)
def f(x):
    #x is an numpy array of shape (2,)
    #x[0] is y and x[1] is z
    #te function returns the value of f(y,z) = (y+3)^2 + sin(y) + (z+1)^2
    y = x[0]
    z = x[1]

    return (y+3)**2 + np.sin(y) + (z+1)**2


#Now we call scipy.optimize.fmin_l_bfgs_b passing the function f and the initial x0 which is a numpy array of values [0,0] and approx_grad = True
x_0 = np.array([0, 0])

#x_min is the minimum point of the function f
#f_min is the value of the function f at the minimum point x_min
#d is a dictionary with information about the optimization process
x_min, f_min, d = opt.fmin_l_bfgs_b(f, x_0, approx_grad=True)

print(f"Mimimum of f(y,z) is at x_min = {x_min}")
print(f"f(y_min,z_min) = f(x_min) = {f_min}")
print(f"Optimization info: {d}")

Mimimum of f(y,z) is at x_min = [-2.57747138 -0.99999927]
f(y_min,z_min) = f(x_min) = -0.3561430123647649
Optimization info: {'grad': array([-1.49324998e-06,  1.46549439e-06]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 21, 'nit': 6, 'warnflag': 0}


We can check the number of times the function $f$ was called with this second approach:

In [11]:
print(f"function f called {d['funcalls']} times")

function f called 21 times


We can also provide an explicit gradient, in this case te function is very simple so rather than approximate it we can just compute the two partial derivatives explicitly:

In [12]:
def f1(x):
    #x is an numpy array of shape (2,)
    #x[0] is y and x[1] is z
    #te function returns the value of f(y,z) = (y+3)^2 + sin(y) + (z+1)^2
    y = x[0]
    z = x[1]

    #provide explicit gradient of f
    y_derivative = 2*(y+3) + np.cos(y)
    z_derivative = 2*(z+1)

    #gradient has shape (2,)
    return (y+3)**2 + np.sin(y) + (z+1)**2, np.array([y_derivative, z_derivative])

#Now we call scipy.optimize.fmin_l_bfgs_b passing the function f and the initial x0 which is a numpy array of values [0,0] and pass the explicitly computed  gradient of f
x_0 = np.array([0, 0])

#x_min is the minimum point of the function f
#f_min is the value of the function f at the minimum point x_min
#d is a dictionary with information about the optimization process
x_min, f_min, d = opt.fmin_l_bfgs_b(f1, x_0)

print(f"Mimimum of f(y,z) is at x_min = {x_min}")
print(f"f(y_min,z_min) = f(x_min) = {f_min}")
print(f"Optimization info: {d}")

Mimimum of f(y,z) is at x_min = [-2.57747137 -0.99999927]
f(y_min,z_min) = f(x_min) = -0.3561430123647611
Optimization info: {'grad': array([-1.50318729e-06,  1.46120529e-06]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 7, 'nit': 6, 'warnflag': 0}


We can check the number of times the function $f$ was called with this second approach:

In [15]:
print(f"function f called {d['funcalls']} times")

function f called 7 times


Whereas with the first approach (i.e. let scipy automatically approximate the gradient) $f$ was called 21 times, in this second case it's called just 7 times, a third! <br>
So, we can say that te automatic numerical approximation of the gradient is significantly more expensive, and the cost becomes
relatively worse when the dimensionality of the domain of f increases.

## Binary logistic regression
It's important to underline that, althougth it's in the name, this is a classification model (like the GGM), and not a regression model (such as linear regression). <br>
We introduce this other model because in this case our goal becomes very different from when we studied the GGM. If, when we implemented and applied the GGM, our goal was to model the distribution of the observed samples $X \mid C$, when using Logistic Regression we want to directly model the class posteriors distribution $C \mid X$. <br>
Using again a generative approach, the Posterior probability for class $h_1$ can be computed from the modeled priors and the modeled class conditional densities, using the Bayes' Theorem:
$$
P(C = h_1 \mid \mathbf{x}) = \frac{f_{\mathbf{X} \mid C}(\mathbf{x} \mid C = h_1) P(C = h_1)}{\sum_{i=0}^{K-1} f_{\mathbf{X} \mid C}(\mathbf{x} | C = c_i) P(C = c_i)} = \frac{f_{\mathbf{X} \mid C}(\mathbf{x} \mid C = h_1) P(C = h_1)}{ f_{\mathbf{X} \mid C}(\mathbf{x} | C = h_0) P(C = h_0) + f_{\mathbf{X} \mid C}(\mathbf{x} | C = h_1) P(C = h_1)}
$$
This can be rewritten as:
$$
P(C = h_1 \mid \mathbf{x}) = \frac{1}{1+ e^{-s(\mathbf{x})}} = \sigma(s(\mathbf{x}))
$$
Where:
- $s(\mathbf{x})$ is the score

- $\sigma (t) = \frac{1}{1+ e^{-t}}$ is called **sigmoid**/logistic function. It has $\lim_{t \to - \infty} \sigma(t) = 0$ and $\lim_{t \to \infty} \sigma(t) = 1$. An important property of this function is that: $1 - \sigma (t) = \sigma (-t)$ <br>
<img src="sigmoid.png" alt="image.png" style="background-color: #ADD8E6; width: 400px; height: 200px;">

For the score, we can use the **log-posterior ratio**, given in the log domain by the sum of the log-likelihood ratio and the prior log odds, as we already did with the GGM:
$$
s(\mathbf{x}) = llr(\mathbf{x}) + \text{log}\frac{\pi}{1+\pi} = \text{log} \frac{f_{\mathbf{X} \mid C}(\mathbf{x} | C = h_1)}{f_{\mathbf{X} \mid C}(\mathbf{x} | C = h_0)} + \text{log}\frac{\pi}{1+\pi}
$$
In this first phase, we can impose and use **linear classification rules**, so the score can be rewritten as:
$$
s(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + b
$$
This expression is linear wrt features $\mathbf{x}$. $\mathbf{w}$ is the weigth vector and is **orthogonal** to the linear decision surface, $b$ is a scalar bias, which also absorbes information from the Priors. $s(\mathbf{x})$ is positive for samples of class $h_1$ and negative for samples of class $h_0$. The equation:
$$
s(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + b = 0
$$
defines the linear decision surface, so the linear hyperplane, which separates our classes. <br>
So, to sum up all of this, we can write:
$$
P(C = h_1 \mid \mathbf{x}, \mathbf{w}, b) = \sigma(\mathbf{w}^T \mathbf{x} + b)
$$
Since we're speaking about probabilities and we just have two classes, we can also write the posterior for the other class, $h_0$, as:
$$
P(C = h_0 \mid \mathbf{x}, \mathbf{w}, b) = 1 - P(C = h_1 \mid \mathbf{x}, \mathbf{w}, b) = 1 - \sigma(\mathbf{w}^T \mathbf{x} + b) = \sigma(- \mathbf{w}^T \mathbf{x} + b)
$$
So, it's clear that we cannot compute these posteriors without knowing the model parameters, $(\mathbf{w}, b)$. <br>
Now, assuming we have a labeled training dataset $\mathcal{D} = [(\mathbf{x}_1, c_1), \ldots, (\mathbf{x}_n, c_n)]$ where classes are independently distributed, we can express the likelihood for the observed labels as
$$
\mathcal{L}(\mathbf{w}, b) = P(C_1 = c_1, \ldots, C_n = c_n | \mathbf{x}_1, \ldots, \mathbf{x}_n, \mathbf{w}, b) = \prod_{i=1}^{n} P(C_i = c_i | \mathbf{x}_i, \mathbf{w}, b)
$$

We can thus apply a **ML approach** to estimate the model parameters that best describe the observed labels $(c_1, \ldots, c_n)$. So we want to find the value of $\mathbf{w}$ and b that maximize the likelihood of our training labels. <br>
We assume that the classes $h_1, h_0$ have labels 1 and 0 respectively. Also, let $y_i = P(C_i = 1 | \mathbf{x}_i, \mathbf{w}, b) = \sigma(\mathbf{w}^T \mathbf{x}_i + b)$. It follows that $P(C_i = 0 | \mathbf{x}_i, \mathbf{w}, b) = 1 - y_i$. <br>
So, the distribution for $C_i | \mathbf{x}_i, \mathbf{w}, b$ is a Bernoulli distribution:

$$
C_i | \mathbf{x}_i, \mathbf{w}, b \sim \text{Ber}(\sigma(\mathbf{w}^T \mathbf{x}_i + b)) = \text{Ber}(y_i)
$$

We can thus rewrite the likelihood using the Bernoulli formula for the density:

$$
\mathcal{L}(\mathbf{w}, b) = \prod_{i=1}^{n} y_i^{c_i} (1 - y_i)^{(1 - c_i)}
$$


As always working with the log-likelihood is more practical, so we have

$$
l(\mathbf{w}, b) = \sum_{i=1}^{n} [c_i \log y_i + (1 - c_i) \log(1 - y_i)] = \sum_{i=1}^{n} [c_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) \ + (1 - c_i) \log \left(1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b) \right)]
$$


Our goal is the maximization of $l$ with respect to $\mathbf{w}$ and $b$.
As briefly said before, we can do this but also follow other two approaches involving the minimization of the average cross-entropy or the minimization of the average Logistic Loss function. <br>
The Average cross-entropy measures how good are the predictions made by te Recognizer, versus the actual labels present in the training data and it's just the negative of the likelihood:
$$
\mathcal{J}(\mathbf{w}, b) = - \sum_{i=1}^{n} [c_i \log y_i + (1 - c_i) \log(1 - y_i)] = - \sum_{i=1}^{n} [c_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) \ + (1 - c_i) \log \left(1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b) \right)]
$$
Te Logistic Loss function is just another way to see all of this, although from an algebric standpoint it's just equivalent to the average cross-entropy after some semplifications and a substitution. As a matter of facts, let:
$$
z_i = 2c_i - 1 \implies
\begin{cases}
    z_i = 2* 0 - 1 = -1 & \text{if } c_i = 0 \\
    z_i = 2 * 1 - 1 = 1  & \text{if } c_i = 1
\end{cases}
$$
If we introduce this substitution and we explicitly write the sigmoid formula we can obtain:
$$
\mathcal{J}(\mathbf{w}, b)
$$
